## MangaDex

In [6]:
! pip install mangadex


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [7]:
import mangadex as md
import os
from dotenv import load_dotenv

load_dotenv()
auth = md.auth.Auth()
auth.login(username = os.environ['USERNAME'], password = os.environ['PASSWORD'], client_id = os.environ['CLIENT_ID'], client_secret = os.environ['CLIENT_SECRET'])
manga = md.series.Manga(auth=auth)
chapter = md.series.Chapter(auth=auth)
customlist = md.series.CustomList(auth=auth)
tag = md.series.Tag()
tag_list = tag.tag_list()
print(tag_list)
cover = md.series.Cover(auth=auth)
scangroup = md.people.ScanlationGroup(auth=auth)
user = md.people.User(auth=auth)
follows = md.people.Follows(auth=auth)

[Tag(tag_id = 0234a31e-a729-4e28-9d6a-3f87c4966b9e, name = {'en': 'Oneshot'}), Tag(tag_id = 07251805-a27e-4d59-b488-f0bfbec15168, name = {'en': 'Thriller'}), Tag(tag_id = 0a39b5a1-b235-4886-a747-1d05d216532d, name = {'en': 'Award Winning'}), Tag(tag_id = 0bc90acb-ccc1-44ca-a34a-b9f3a73259d0, name = {'en': 'Reincarnation'}), Tag(tag_id = 256c8bd9-4904-4360-bf4f-508a76d67183, name = {'en': 'Sci-Fi'}), Tag(tag_id = 292e862b-2d17-4062-90a2-0356caa4ae27, name = {'en': 'Time Travel'}), Tag(tag_id = 2bd2e8d0-f146-434a-9b51-fc9ff2c5fe6a, name = {'en': 'Genderswap'}), Tag(tag_id = 2d1f5d56-a1e5-4d0d-a961-2193588b08ec, name = {'en': 'Loli'}), Tag(tag_id = 31932a7e-5b8e-49a6-9f12-2afa39dc544c, name = {'en': 'Traditional Games'}), Tag(tag_id = 320831a8-4026-470b-94f6-8353740e6f04, name = {'en': 'Official Colored'}), Tag(tag_id = 33771934-028e-4cb3-8744-691e866a923e, name = {'en': 'Historical'}), Tag(tag_id = 36fd93ea-e8b8-445e-b836-358f02b3d33d, name = {'en': 'Monsters'}), Tag(tag_id = 391b0423-d8

## One Piece

In [8]:
manga_list = manga.get_manga_list(title = "One Piece", limit = 1)
one_piece_manga = manga_list[0]
one_piece_id = one_piece_manga.manga_id

print(one_piece_manga)
print(one_piece_id)
one_piece_feed = one_piece_manga.manga_feed(one_piece_id)
print(len(one_piece_feed))
chapter = one_piece_feed[0]
chapter_list = chapter.get_chapter_list()
# print(chapter_list)

chapter_ids = [chp.chapter_id for chp in one_piece_feed]

Manga(manga_id = a1c7c817-4e59-43b7-9365-09675a149a6f, title = {'en': 'One Piece'},                 altTitles = [{'ja': 'ワンピース'}, {'uk': 'Ван Піс'}, {'ru': 'One Piece. Большой куш'}, {'ru': 'Ван Пис'}, {'fa': 'وان پیس'}, {'hi': 'वन पीस'}, {'bn': 'ওয়ান পিস'}, {'ne': 'वान पिस'}, {'ne': 'वान पीस'}, {'ta': 'ஒன் பீஸ்'}, {'th': 'วันพีซ'}, {'mn': 'Уан Пийс'}, {'kk': 'Уан Пис'}, {'kk': 'One Piece. Батпан құйрық'}, {'kk': 'Бір бөлік'}, {'he': 'וואן פיס'}, {'ar': 'ون بيس'}, {'ar': 'قطعة واحدة'}, {'zh': '海贼王'}, {'zh': '海盗路飞'}, {'zh': '航海王'}, {'zh-hk': '海賊王'}, {'my': 'ဝမ်းပိစ်'}, {'vi': 'Đảo Hải Tặc'}, {'vi': 'Vua Hải Tặc'}, {'ms': 'Budak Getah'}, {'ko': '원피스'}, {'te': 'వన్ పీస్'}], description = {'en': 'Gol D. Roger, a man referred to as the "Pirate King," is set to be executed by the World Government. But just before his demise, he confirms the existence of a great treasure, One Piece, located somewhere within the vast ocean known as the Grand Line. Announcing that One Piece can be claimed by a

## Wrapper docs seems outdated, gna just refer to official docs

In [11]:
import requests
import os
import time

def download_chapter_images(chapter_id, output_directory=None, use_data_saver=True):
    """
    Download all images for a given MangaDex chapter ID.
    
    Args:
        chapter_id (str): The MangaDex chapter ID
        output_directory (str, optional): Directory to save images. Defaults to chapter ID.
        use_data_saver (bool, optional): Whether to use compressed images. Defaults to False.
    
    Returns:
        list: Paths to downloaded images
    """
    # Create output directory if it doesn't exist
    if output_directory is None:
        output_directory = f"chapter_{chapter_id}"
    
    os.makedirs(output_directory, exist_ok=True)
    
    # Step 1: Get the chapter's image delivery metadata
    metadata_url = f"https://api.mangadex.org/at-home/server/{chapter_id}"
    response = requests.get(metadata_url)
    
    if response.status_code != 200:
        raise Exception(f"Failed to get chapter metadata: {response.status_code} - {response.text}")
    
    metadata = response.json()
    base_url = metadata["baseUrl"]
    chapter_hash = metadata["chapter"]["hash"]
    
    # Choose between original quality or compressed images
    if use_data_saver:
        images = metadata["chapter"]["dataSaver"]
        quality_mode = "data-saver"
    else:
        images = metadata["chapter"]["data"]
        quality_mode = "data"
    
    downloaded_paths = []
    
    # Step 2: Construct URLs and download images
    for i, image in enumerate(images):
        # Construct the full image URL
        image_url = f"{base_url}/{quality_mode}/{chapter_hash}/{image}"
        
        # Determine file extension from the image filename
        _, ext = os.path.splitext(image)
        
        # Create a clean filename with page number and original extension
        output_filename = f"page_{i+1:03d}{ext}"
        output_path = os.path.join(output_directory, output_filename)
        
        # Download the image
        print(f"Downloading image {i+1}/{len(images)}: {output_filename}")
        image_response = requests.get(image_url, headers={
            # Do NOT include authentication headers as per documentation
            "Referer": "https://mangadex.org/"  # Setting referer to be polite
        })
        
        if image_response.status_code != 200:
            print(f"Failed to download image {i+1}: {image_response.status_code}")
            continue
        
        # Save the image to disk
        with open(output_path, "wb") as f:
            f.write(image_response.content)
        
        downloaded_paths.append(output_path)
        
        # Add a small delay to be nice to the server
        time.sleep(0.5)
    
    print(f"Downloaded {len(downloaded_paths)} images to {output_directory}")
    return downloaded_paths

def download_chapters(chapter_ids, base_output_dir="manga_downloads", use_data_saver=False):
    """
    Download multiple chapters by their IDs
    
    Args:
        chapter_ids (list): List of chapter IDs to download
        base_output_dir (str, optional): Base directory for downloads
        use_data_saver (bool, optional): Whether to use compressed images
    """
    os.makedirs(base_output_dir, exist_ok=True)
    
    for chapter_id in chapter_ids:
        chapter_dir = os.path.join(base_output_dir, f"chapter_{chapter_id}")
        print('chapter_dir:', chapter_dir)
        os.makedirs(chapter_dir, exist_ok=True)
        try:
            download_chapter_images(
                chapter_id=chapter_id,
                output_directory=chapter_dir,
                use_data_saver=use_data_saver
            )
            print(f"Successfully downloaded chapter {chapter_id}")
        except Exception as e:
            print(f"Error downloading chapter {chapter_id}: {str(e)}")

In [13]:
download_chapters(chapter_ids)
# for chapter_id in chapter_ids:
#     try:
#         download_chapter_images(chapter_id, output_directory="one_piece_chapter")
#     except Exception as e:
#         print('Failed to download chapter', chapter_id, e)
#         continue

chapter_dir: manga_downloads/chapter_29ea1332-8840-431c-a211-cdf811a7673a
Error downloading chapter 29ea1332-8840-431c-a211-cdf811a7673a: Failed to get chapter metadata: 404 - {"result":"error","errors":[{"id":"770a6444-c695-570d-bfd2-a0eec2e95ce9","status":404,"title":"not_found_http_exception","detail":"Chapter with ID 29ea1332-8840-431c-a211-cdf811a7673a not found.","context":null}]}
chapter_dir: manga_downloads/chapter_03cfac12-4efe-4faf-8ed9-1a08a4efe0b2
Error downloading chapter 03cfac12-4efe-4faf-8ed9-1a08a4efe0b2: Failed to get chapter metadata: 404 - {"result":"error","errors":[{"id":"770a6444-c695-570d-bfd2-a0eec2e95ce9","status":404,"title":"not_found_http_exception","detail":"Chapter with ID 03cfac12-4efe-4faf-8ed9-1a08a4efe0b2 not found.","context":null}]}
chapter_dir: manga_downloads/chapter_5faea764-83dc-403e-b92e-d2c9028f1b2c
Error downloading chapter 5faea764-83dc-403e-b92e-d2c9028f1b2c: Failed to get chapter metadata: 404 - {"result":"error","errors":[{"id":"770a6444-